<a href="https://colab.research.google.com/github/MessaoudAbbas/Codes_Military_Objects/blob/main/svm_autism_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

# # Then move kaggle.json into the folder where the API expects to find it.

!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!mkdir /content/data

In [ ]:
!kaggle competitions download -c abide

 99% 881M/886M [00:12<00:00, 54.3MB/s]
100% 886M/886M [00:12<00:00, 72.1MB/s]


In [ ]:
!unzip "/content/abide.zip" -d "/content/data/"

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: /content/data/comp_data/train/rois_aal/Trinity_0050263_rois_aal.1D  
  inflating: /content/data/comp_data/train/rois_aal/Trinity_0050264_rois_aal.1D  
  inflating: /content/data/comp_data/train/rois_aal/Trinity_0050265_rois_aal.1D  
  inflating: /content/data/comp_data/train/rois_aal/Trinity_0050266_rois_aal.1D  
  inflating: /content/data/comp_data/train/rois_aal/Trinity_0050267_rois_aal.1D  
  inflating: /content/data/comp_data/train/rois_aal/Trinity_0050268_rois_aal.1D  
  inflating: /content/data/comp_data/train/rois_aal/Trinity_0050269_rois_aal.1D  
  inflating: /content/data/comp_data/train/rois_aal/Trinity_0050270_rois_aal.1D  
  inflating: /content/data/comp_data/train/rois_aal/Trinity_0050271_rois_aal.1D  
  inflating: /content/data/comp_data/train/rois_aal/Trinity_0051132_rois_aal.1D  
  inflating: /content/data/comp_data/train/rois_aal/Trinity_0051134_rois_aal.1D  
  inflating: /conten

In [ ]:
import h5py
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
import time
import sklearn


In [ ]:
pheno_train = pd.read_csv('/content/data/comp_data/pheno_train.csv')
pheno_test = pd.read_csv('/content/data/comp_data/pheno_test.csv')
ss = pd.read_csv('/content/data/comp_data/sample_submission.csv')
func = h5py.File('/content/data/comp_data/abide.hdf5', 'r')

# adjust according to submission format
pheno_train['DX_GROUP'] = pheno_train['DX_GROUP'] - 1.0


In [ ]:
def get_data_train(data, pheno, derivative):
    X = []
    y = []
    i = 0
    total = pheno.shape[0]
    for row in pheno.iterrows():
        file_id, dx_group = row[1]['FILE_ID'], row[1]['DX_GROUP']
        connectivity = data['patients'][file_id][derivative][()]
        X.append(connectivity)
        y.append(dx_group)
        sys.stdout.write("\r{:.2f}%>".format(i/total))
        sys.stdout.flush()
        i += 1

    X = np.array(X).astype(np.float32)
    y = np.array(y).astype(np.float32)
    return X, y

def get_data_test(data, pheno, derivative):
    X_test = []
    sub_ids = []
    j = 0
    total = pheno.shape[0]
    for row in pheno.iterrows():
        file_id, sub_id = row[1]['FILE_ID'], row[1]['SUB_ID']
        connectivity = data['patients'][file_id][derivative][()]
        X_test.append(connectivity)
        sub_ids.append(sub_id)
        sys.stdout.write("\r{:.2f}%>".format(j/total))
        sys.stdout.flush()
        j += 1

    X_test = np.array(X_test).astype(np.float32)
    return X_test, sub_ids


In [ ]:
X, y = get_data_train(func, pheno_train, 'aal')
X.shape, y.shape


1.00%>

((931, 6670), (931,))

In [ ]:
X_test, sub_ids = get_data_test(func, pheno_test, 'aal')
X_test.shape, len(sub_ids)


0.99%>

((104, 6670), 104)

# **Hist Gradient Boosting Classifier**

In [ ]:
%%time
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier as hgbc
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import gc

N_SPLITS = 10
kf = KFold(n_splits=N_SPLITS)
final_preds = np.zeros((X_test.shape[0], N_SPLITS))
i = 0
for tr_idx, val_idx in kf.split(X):
    clf = hgbc(max_depth=6, max_leaf_nodes=25, verbose=0, max_iter=30)
    clf.fit(X[tr_idx], y[tr_idx])
    final_preds[:, i] = clf.predict_proba(X_test)[:, 1]
    fold_preds = clf.predict_proba(X[val_idx])[:, 1]
    print('Fold: {0}, score: {1}'.format(i, roc_auc_score(y[val_idx], fold_preds)))
    del clf
    gc.collect()
    i += 1


/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


Fold: 0, score: 0.6535326086956522
Fold: 1, score: 0.6932573599240266
Fold: 2, score: 0.6077705827937097
Fold: 3, score: 0.6808905380333952
Fold: 4, score: 0.7437258687258688
Fold: 5, score: 0.651031894934334
Fold: 6, score: 0.7078703703703703
Fold: 7, score: 0.7574074074074074
Fold: 8, score: 0.6875
Fold: 9, score: 0.6796296296296298
CPU times: user 6min 40s, sys: 1min 55s, total: 8min 36s
Wall time: 5min 50s


In [ ]:
from sklearn.neighbors import KNeighborsClassifier


In [ ]:


N_SPLITS = 10
kf = KFold(n_splits=N_SPLITS)
final_preds = np.zeros((X_test.shape[0], N_SPLITS))
i = 0
for tr_idx, val_idx in kf.split(X):
    #clf = hgbc(max_depth=6, max_leaf_nodes=25, verbose=0, max_iter=30)
    clf= KNeighborsClassifier()
    clf.fit(X[tr_idx], y[tr_idx])
    final_preds[:, i] = clf.predict_proba(X_test)[:, 1]
    fold_preds = clf.predict_proba(X[val_idx])[:, 1]
    print('Fold: {0}, score: {1}'.format(i, roc_auc_score(y[val_idx], fold_preds)))
    del clf
    gc.collect()
    i += 1


Fold: 0, score: 0.6064311594202898
Fold: 1, score: 0.6236942070275404
Fold: 2, score: 0.626040703052729
Fold: 3, score: 0.5134508348794062
Fold: 4, score: 0.6877413127413127
Fold: 5, score: 0.5361163227016885
Fold: 6, score: 0.6189814814814815
Fold: 7, score: 0.5333333333333334
Fold: 8, score: 0.6113425925925926
Fold: 9, score: 0.656712962962963


In [ ]:
from sklearn.tree import DecisionTreeClassifier


In [ ]:
N_SPLITS = 10
kf = KFold(n_splits=N_SPLITS)
final_preds = np.zeros((X_test.shape[0], N_SPLITS))
i = 0
for tr_idx, val_idx in kf.split(X):
    #clf = hgbc(max_depth=6, max_leaf_nodes=25, verbose=0, max_iter=30)
    clf= DecisionTreeClassifier(max_depth =2, random_state = 42)
    clf.fit(X[tr_idx], y[tr_idx])
    final_preds[:, i] = clf.predict_proba(X_test)[:, 1]
    fold_preds = clf.predict_proba(X[val_idx])[:, 1]
    print('Fold: {0}, score: {1}'.format(i, roc_auc_score(y[val_idx], fold_preds)))
    del clf
    gc.collect()
    i += 1


Fold: 0, score: 0.5285326086956521
Fold: 1, score: 0.5712250712250713
Fold: 2, score: 0.5282146160962071
Fold: 3, score: 0.5807050092764379
Fold: 4, score: 0.5868725868725868
Fold: 5, score: 0.577626641651032
Fold: 6, score: 0.5511574074074074
Fold: 7, score: 0.5884259259259259
Fold: 8, score: 0.5356481481481481
Fold: 9, score: 0.5002314814814816


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

model = Sequential()



In [ ]:
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1))



In [ ]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
model.fit(X, y,batch_size=128,epochs=50,verbose=1, validation_data=(X, y))


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


8/8 [==============================] - 5s 499ms/step - loss: 6.1077e-08 - accuracy: 0.4930 - val_loss: 6.1077e-08 - val_accuracy: 0.4876
Epoch 2/50
8/8 [==============================] - 4s 538ms/step - loss: 6.1077e-08 - accuracy: 0.4909 - val_loss: 6.1077e-08 - val_accuracy: 0.4876
Epoch 3/50
8/8 [==============================] - 4s 471ms/step - loss: 6.1077e-08 - accuracy: 0.4855 - val_loss: 6.1077e-08 - val_accuracy: 0.4876
Epoch 4/50
8/8 [==============================] - 3s 412ms/step - loss: 6.1077e-08 - accuracy: 0.4909 - val_loss: 6.1077e-08 - val_accuracy: 0.4876
Epoch 5/50
8/8 [==============================] - 4s 454ms/step - loss: 6.1077e-08 - accuracy: 0.4952 - val_loss: 6.1077e-08 - val_accuracy: 0.4876
Epoch 6/50
8/8 [==============================] - 4s 570ms/step - loss: 6.1077e-08 - accuracy: 0.4887 - val_loss: 6.1077e-08 - val_accuracy: 0.4876
Epoch 7/50
8/8 [==============================] - 4s 455ms/step - loss: 6.1077e-08 - accuracy: 0.4909 - val_loss: 6.1077e-0

# LSTM

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from time import perf_counter

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from time import perf_counter



models = {
    "KNN": {"model": KNeighborsClassifier(), "perf": 0},
    "SVM (RBF)": {"model": SVC(), "perf": 0}
}

for name, model_info in models.items():
    start = perf_counter()
    model_info['model'].fit(X, y)
    duration = perf_counter() - start
    duration = round(duration, 2)
    model_info["perf"] = duration
    print(f"{name:20} trained in {duration} sec")

    if "SVM" in name:
        # Add cross-validation for SVM
        cv_start = perf_counter()
        scores = cross_val_score(model_info['model'], X, y, cv=5)  # Adjust the number of folds as needed
        cv_duration = round(perf_counter() - cv_start, 2)
        print(f"{name} cross-validation accuracy: {np.mean(scores):.4f} (CV time: {cv_duration} sec)")


KNN                  trained in 0.03 sec
SVM (RBF)            trained in 6.07 sec
SVM (RBF) cross-validation accuracy: 0.6671 (CV time: 22.2 sec)


In [ ]:
models_acc = []
for name, model_info in models.items():
    models_acc.append([name, model_info["model"].score(X, y), model_info["perf"]])


In [ ]:
df_acc = pd.DataFrame(models_acc)
df_acc.columns = ['Model', 'Accuracy w/o scaling', 'Training time (sec)']
df_acc.sort_values(by = 'Accuracy w/o scaling', ascending = False, inplace=True)
df_acc.reset_index(drop = True, inplace=True)
df_acc

,Model,Accuracy w/o scaling,Training time (sec)
0,SVM (RBF),0.949517,6.07
1,KNN,0.744361,0.03
